## PL2 - Phuture Layer 2 Index - Methodology Template

## Configuring packages

In [1]:
import pandas as pd
import decouple
import sys
sys.path.append('../')
from methodology import MethodologyBase
from pycoingecko import CoinGeckoAPI
key =  decouple.config("CG_KEY")
cg = CoinGeckoAPI(api_key=key)

## Index Variables

In [2]:
min_mcap =  0.5e8
min_weight = 0.005
max_weight = 0.3
circ_supply_threshold = 0.05
liveness_threshold = 1 # In days
liquidity_consistency = 90 # In days
coingecko_category = ["layer-2"]

# Calculating max slippage based on liquidity threshold
liq = 2*1e6
liq_os = liq / 2 # one-side liquidity 
trade_value_tiny = 1e2
trade_value = 1e5
max_slippage = (liq_os + trade_value_tiny) / (liq_os + trade_value) - 1
# round to the nearest .01
max_slippage = round(max_slippage, 2)
print(max_slippage)

# Dataframes to remove
stablecoins = pd.DataFrame(cg.get_coins_markets('usd',category='stablecoins')).set_index('id')
ctokens = pd.DataFrame(cg.get_coins_markets('usd',category='compound-tokens')).set_index('id')
atokens = pd.DataFrame(cg.get_coins_markets('usd',category='aave-tokens')).set_index('id')

methodology_class = MethodologyBase(
                                'arbitrum-one',
                                min_mcap,
                                min_weight,
                                max_weight,
                                65535,
                                circ_supply_threshold,
                                liveness_threshold,
                                liquidity_consistency,
                                max_slippage,
                                coingecko_category)

-0.09


### Token inclusion criteria

1. **Project and token characteristics**
    1. The project’s token should have been listed on CoinGecko with pricing data spanning at least 6 months prior to the date of inclusion in the index.
    1. The project should have an active community.
    1. No rebasing or deflationary tokens.
    1. Synthetic tokens which derive their value from external price feeds are not permissible. 
    1. The project’s token must not have the ability to pause token transfers.
    1. The project must have a minimum circulating market cap of $50mm.
   <br>
1. **Pricing requirmeents**
    1. Token should have a reliable price feed from one of our authorised providers. 
    <br>
1. **Token Supply requirements**
    1. The project's token must have a circulating supply greater than 20% of the max supply. In cases where a token does not have a max supply, the minting mechanics would need to be assessed.
    <br>
1. **Liquidity Requirements**
    1. The token must be listed on a supported exchange.
    1. The price should experience no more than 12% price impact when executing a $100k trade.
    <br>
1. **Security Requirements**
    1. The project must have been audited by smart contract security professionals with the audit report(s) publicly available. Alternatively, the protocol must have been operating long enough to create a consensus about its safety in the decentralised finance community.
1. **Weighting requirements**
    1. The maximum weight any one token can have is 30%.
    1. All excess weight is proportionally redistributed to all uncapped tokens. After this has been completed, if another token now exceeds the 30% threshold the excess will be redistributed to the remaining uncapped tokens. This process will occur iteratively until there are no tokens that exceed the maximum weight.
    1. Any asset with a weight below 0.5% will be removed from the index.


## Run Methodology

In [3]:

results, liquidity = methodology_class.main(
    df_to_remove=[stablecoins,ctokens,atokens],
    remove_category_assets=['gelato']
    )
liquidity

0x0000000000000000000000000000000000001010 polygon-pos
0x6b289cceaa8639e3831095d75a3e43520fabf552 avalanche
0xec6adef5e1006bb305bb1975333e8fc4071295bf optimistic-ethereum
0x319f865b287fcc10b30d8ce6144e8b6d1b476999 arbitrum-one
ethereum      -0.012638
polygon-pos   -0.002226
Name: matic-network, dtype: float64
arbitrum-one   -0.002327
ethereum       -0.029000
Name: arbitrum, dtype: float64
ethereum   -0.023281
Name: immutable-x, dtype: float64
optimistic-ethereum   -0.008001
Name: optimism, dtype: float64
ethereum   -0.012928
Name: mantle, dtype: float64
ethereum   -0.05178
Name: starknet, dtype: float64
ethereum   -0.031629
Name: metis-token, dtype: float64
Series([], Name: loopring, dtype: float64)
Series([], Name: skale, dtype: float64)
Series([], Name: boba-network, dtype: float64)
ethereum   -0.061965
Name: cartesi, dtype: float64
ethereum   -0.036714
Name: satoshivm, dtype: float64
Series([], Name: cocos-bcx, dtype: float64)
ethereum   -0.023355
Name: hytopia, dtype: float64
Serie

,arbitrum-one,binance-smart-chain,ethereum,optimistic-ethereum,polygon-pos,optimal chain,best slippage,best slippage chain
id,,,,,,,,
matic-network,NaN,-0.234648,-0.012638,NaN,-0.002226,polygon-pos,-0.002226,polygon-pos
arbitrum,-0.002327,NaN,-0.029000,NaN,NaN,arbitrum-one,-0.002327,arbitrum-one
optimism,NaN,NaN,NaN,-0.008001,NaN,optimistic-ethereum,-0.008001,optimistic-ethereum
mantle,NaN,NaN,-0.012928,NaN,NaN,ethereum,-0.012928,ethereum
immutable-x,NaN,NaN,-0.023281,NaN,NaN,ethereum,-0.023281,ethereum
hytopia,NaN,NaN,-0.023355,NaN,NaN,ethereum,-0.023355,ethereum
metis-token,NaN,NaN,-0.031629,NaN,NaN,ethereum,-0.031629,ethereum
satoshivm,NaN,NaN,-0.036714,NaN,NaN,ethereum,-0.036714,ethereum
starknet,NaN,NaN,-0.051780,NaN,NaN,ethereum,-0.051780,ethereum


In [4]:
results

,symbol,market_cap,weight,weight_converted,address,blockchain
id,,,,,,
matic-network,MATIC,9.534558e+09,0.300000,19661,0x0000000000000000000000000000000000001010,polygon-pos
arbitrum,ARB,4.356498e+09,0.184722,12106,0x912ce59144191c1204e64559fe8253a0e49e6548,arbitrum-one
immutable-x,IMX,3.942346e+09,0.167161,10955,0xf57e7e7c23978c3caec3c3548e3d615c346e79ff,ethereum
optimism,OP,3.340029e+09,0.141622,9282,0x4200000000000000000000000000000000000042,optimistic-ethereum
mantle,MNT,2.701086e+09,0.114530,7506,0x3c3a81e81dc49a522a592e7622a7e711c06bf354,ethereum
starknet,STRK,1.418740e+09,0.060157,3942,0xca14007eff0db1f8135f4c25b34de49ab0d42766,ethereum
metis-token,METIS,5.110032e+08,0.021667,1419,0x9e32b13ce7f2e80a01932b42553652e053d6ed8e,ethereum
cartesi,CTSI,2.391417e+08,0.010140,664,0x491604c0fdf08347dd1fa4ee062a822a5dd06b5d,ethereum
